# From Features to Forecasts: Predicting Home Prices Using Computational Statistics
Angeliki Andreadi, Laila Ibrahim, Salsabil Mtiraoui


## 1. Library Imports

In [ ]:
# General use
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import io
import sys

# Preprossessing/Variables selection
import scipy.stats as stats
from scipy.stats import pearsonr
from scipy.stats import f_oneway

from sklearn.preprocessing import LabelEncoder
from itertools import combinations
import statsmodels.formula.api as smf

# Time Series
from statsmodels.stats.anova import anova_lm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

## 2. Data Loading

### 2.1 To execute on VSCode

In [ ]:
# Uploaded csv => train.csv

sys.path.append("house-prices-advanced-regression-techniques")
df = pd.read_csv('house-prices-advanced-regression-techniques/train.csv')

### 2.2  To execute on Google Colab

In [ ]:
# Uploaded file => train.csv

from google.colab import files
uploaded = files.upload()

# Automatically get the correct filename, no matter what it's called
filename = next(iter(uploaded))
print(f"Using file: {filename}")

# Read the uploaded CSV file
df = pd.read_csv(io.BytesIO(uploaded[filename]))

## 3. Pre-prossessing

In [ ]:
# Drop the Id column since we don't need it
df = df.drop('Id', axis=1)

# We will replace NaN with "none",to make it easier to work with it
df['Alley'] = df['Alley'].fillna('NA')

df.head(3)

### 3.1 Separating each data type

In [ ]:
# Compute number of unique values for each column of df
n_unique = df.nunique()
# Determines if data is binary or categorical

# Detrieves data type for each col in df
dtypes = df.dtypes

# Threshold after which the data is considered quantitative
quant_threshold = 2

# Variables where each sorted column's name will be stocked
quant_var = []
binary_var = []
categorical_var = []


for col in df.columns:
  if dtypes[col] in ['int64', 'float64']:
    if n_unique[col] == 2:
      # If column's datatype is numbers and has only two possible values
      binary_var.append(col)

    elif n_unique[col]>quant_threshold:
      # If column's datatype is numbers but has more than quant_threshold
      quant_var.append(col)
    else:
      categorical_var.append(col)

  elif dtypes[col] in ['object', 'category', 'bool']:
        if n_unique[col] == 2:
            if df[col].dtype == 'object':
                # If the column is an object type but only has two values, convert it to binary
                df[col] = df[col].map({df[col].unique()[0]: 0, df[col].unique()[1]: 1})
            binary_var.append(col)
        else:
            categorical_var.append(col)

# New data frames for each type

df_quant = df[quant_var] # Quantitative variables
df_binary = df[binary_var] # Binary variables
df_categorical = df[categorical_var] # Categorical variables

print("Quantitative variables:", quant_var)
print("Binary variables:", binary_var)
print("Categorical variables:", categorical_var)

# Checking that all our variables are here
print(len(quant_var) + len(binary_var) + len(categorical_var))


### 3.2 Dropping incomplete columns

In [ ]:
# Quantitative

# Total count for each variable
total_counts_quant = df_quant.apply(lambda x: x.value_counts().sum())

# Keep only the variables with a total count of 1460
df_quant = df_quant.loc[:, total_counts_quant == 1460]
df_quant

In [ ]:
# Categorical

# Total count for each variable
total_counts = df_categorical.apply(lambda x: x.value_counts().sum())

# Keep only the variables with a total count of 1460
filtered_cat = df_categorical.loc[:, total_counts == 1460]

# From now, we'll work with this instead of df_categorical
filtered_cat

## 4. Feature Selection

### 4.1 Quantitative variables

In [ ]:
def descriptive_stats(df_quand):
  # Summary for each variable
  stats_summary = df_quant.describe().T

  for col in df_quant.columns:
    mean = df_quant[col].mean()
    sd = df_quant[col].std()
    n = df_quant[col].count()
    CI = stats.t.interval(0.95, n-1, loc = mean, scale= sd/np.sqrt(n))

    stats_summary.loc[col, "CI_95%_lower"] = CI[0]
    stats_summary.loc[col, "CI_95%_upper"] = CI[1]

  return stats_summary


def hypothesis_test(def_quant, price):
  res = {}

  for col in df_quant.columns:
    if col == price:
      continue


    corr, p_value = pearsonr(df_quant[col], df_quant[price])
    res[col] = {"Correlation coeff" : corr, "P-value": p_value}

  return res

stats_summary = descriptive_stats(df_quant)
stats_summary

hypothesis_test_results = hypothesis_test(df_quant, "SalePrice")
hypothesis_test_results = pd.DataFrame(hypothesis_test_results).T  # Transpose to get variables as index
hypothesis_test_results.index.name = "Variable"
hypothesis_test_results.columns = ["Correlation Coefficient", "P-value"]
hypothesis_test_results
# Display the table

### 4.1.2 Quantitative variable ranking


In [ ]:
def rank_features(df_quant, target_variable):
    stats_summary = descriptive_stats(df_quant)
    hypothesis_results = hypothesis_test(df_quant, target_variable)

    # Convert hypothesis test results to Df for easier manipulation
    hypothesis_df = pd.DataFrame(hypothesis_results).T

    # Combine both descriptive stats and hypothesis test results into one DataFrame
    combined_df = stats_summary.join(hypothesis_df)
    # We will rank by correlation, p-value, and the variance

    # Rank by correlation coefficient (absolute value to consider positive and negative correlations)
    combined_df['abs_corr'] = combined_df['Correlation coeff'].abs()

    # Add a score based on the variance (higher variance = more spread = more informative)
    combined_df['variance'] = combined_df['std'].rank(ascending=False, method='min')

    # Add a score based on p-value (lower p-value = higher significance)
    combined_df['p_value_rank'] = combined_df['P-value'].rank(ascending=True, method='min')

    combined_df['final_rank'] = combined_df['p_value_rank']

    # Final rank score (each factor is kind of arbirtarily weighted, the more * the more important it is considered)
    #combined_df['final_rank'] = combined_df['abs_corr'] * 10 + combined_df['p_value_rank'] * 2 + combined_df['variance'] * 5


    combined_df_sorted = combined_df.sort_values(by='final_rank', ascending=True)

    return combined_df_sorted[['Correlation coeff', 'P-value', 'CI_95%_lower', 'CI_95%_upper', 'variance', 'final_rank']]

# Example usage (assuming df_quant is your quantitative DataFrame and "SalePrice" is the target variable)
ranked_features = rank_features(df_quant, "SalePrice")
print(ranked_features)

### 4.2 Binary variables

In [ ]:
# Descriptive stats for binary variables
desc_stats_binary = df_binary.describe().T
print(desc_stats_binary)

# Correlation with SalePrice for binary variables
df_binary.corrwith(df["SalePrice"]).sort_values(ascending=False)

#### 4.2.1 $2^k$ factorial design

In [ ]:
factors = ["Street", "Utilities", "CentralAir"]
df_factors = df[factors + ["SalePrice"]].dropna()

# Build formula with all main effects + 2-way + 3-way interactions
main_effects = "+".join(factors)
interactions = "+".join([":".join(combo) for i in range(2, 4) for combo in combinations(factors, i)])
formula = f"SalePrice ~ {main_effects} + {interactions}"

# Fit linear model
model = smf.ols(formula=formula, data=df_factors).fit()
print(model.summary())


### 4.3 Categorical variables

### 4.3.Feature selection

In [ ]:
# Descriptive statistics for categorical variables
df_categorical.describe()

# For the rest of the project, we will work with 'filtered_cat', which represents our filtered categorical variables

In [ ]:
def rank_categorical_vars(df, filtered_cat, prices):
  results = []
  # Loop through each cat var
  for col in filtered_cat:
    try:
      # Build list of arrays, each array is Sale Price for on category of the cat variable.
      # Ex. makes array for prices roofStyle == "Flat"
      groups = [df[prices][filtered_cat[col]== cat].dropna() for cat in filtered_cat[col].unique()]

      if len(groups) > 1:
        # One way anova since we have one quant and several categorical
          f_stat, p_val = f_oneway(*groups)
          # * passes each item as a separate argument

          # Compute diff between highest and lowest mean across the categories, to sense how big the effect is
          mean_range = max([g.mean() for g in groups]) - min([g.mean() for g in groups])
          results.append({
              "Variable": col,
              "F-value": f_stat,
              "P_value": p_val,
              "Mean_range": mean_range
          })
    except Exception as e:
        print(f"Could not process {col}: {e}")
        continue
  df_results = pd.DataFrame(results)
  df_results["p_rank"] = df_results["P_value"].rank(ascending=True)
  df_results["range_rank"] = df_results["Mean_range"].rank(ascending=False)

    # Arbitrary scoring: lower p, higher range = better
  df_results["final_score"] = df_results["p_rank"] * 2 + df_results["range_rank"]
  # lower score = better variable, bcs lower p rank = + significant p-value and high range rank = large spread between group means
  df_results = df_results.sort_values(by="final_score")#.rank(ascending=False)

  return df_results

categorical_rankings = rank_categorical_vars(df, filtered_cat, "SalePrice")
print(categorical_rankings)

# Sort by p-value for cleaner visualization
df_sorted = categorical_rankings.sort_values("P_value")

plt.figure(figsize=(12, 6))
sns.barplot(data=df_sorted, x="Variable", y="P_value", palette="viridis")

plt.axhline(0.05, color='red', linestyle='--', label='Significance threshold (0.05)')
plt.xticks(rotation=90)
plt.ylabel("P-value")
plt.title("P-values from ANOVA for Categorical Variables")
plt.legend()
plt.tight_layout()
plt.show()

#
df_sorted = categorical_rankings.sort_values("Mean_range", ascending=False)

plt.figure(figsize=(12, 6))
sns.barplot(data=df_sorted, x="Variable", y="Mean_range", palette="coolwarm")
plt.xticks(rotation=90)
plt.ylabel("Range of Means")
plt.title("Range of SalePrice Means Across Categories")
plt.tight_layout()
plt.show()

# Sort so that top-ranked variables are on top of the plot
df_sorted = categorical_rankings.sort_values("final_score", ascending=True)

plt.figure(figsize=(12, 6))
sns.barplot(data=df_sorted, x="final_score", y="Variable", palette="viridis")
plt.xlabel("Final Score")
plt.ylabel("Categorical Variable")
plt.title("Ranking of Categorical Variables Based on Final Score")
plt.tight_layout()
plt.show()


# Low P-values means the categorical value likely affects the SalesPrice, it is not due to chance


In [ ]:

import statsmodels.api as sm

top_cat_vars = categorical_rankings.sort_values("final_score").head(10)["Variable"].tolist()

#  One-hot encoding
df_encoded_cat = pd.get_dummies(filtered_cat[top_cat_vars], drop_first=True).astype('float64')

#  Ajouter la constante (intercept)
X_cat = sm.add_constant(df_encoded_cat)

#  Préparer la variable cible
y = df["SalePrice"].astype('float64')

#  Nettoyage — garder uniquement les lignes sans NaN
X_cat, y = X_cat.align(y, join='inner', axis=0)

# Régression linéaire
model_cat = sm.OLS(y, X_cat).fit()

print(model_cat.summary())

In [ ]:
# Sort coefficients outside the constant and keep only the most significant ones
coeffs = model_cat.params.drop("const").sort_values(key=abs, ascending=False).head(20)  # top 20 coeffs en valeur absolue

# We shorten the too long names
labels = [label if len(label) <= 25 else label[:22] + "..." for label in coeffs.index]

plt.figure(figsize=(12, 10))
sns.barplot(x=coeffs.values, y=labels, palette="coolwarm")
plt.axvline(x=0, color='black', linestyle='--')
plt.title("Top 20 modalités catégorielles impactant le prix de vente", fontsize=14)
plt.xlabel("Coefficient estimé (variation du prix)", fontsize=12)
plt.ylabel("Modalités", fontsize=12)
plt.grid(axis='x', linestyle=':', alpha=0.6)
plt.tight_layout()
plt.show()


In [ ]:
# List of quantitative variables by correlation and p-value
selected_quant_vars = [
    "OverallQual",
    "GrLivArea",
    "GarageCars",
    "GarageArea",
    "TotalBsmtSF",
    "1stFlrSF",
    "FullBath",
    "TotRmsAbvGrd",
    "YearBuilt",
    "YearRemodAdd"
]

# Sub-dataframe with these variables only
df_selected_quant = df[selected_quant_vars].copy()


# Selection of the top 10 most relevant categorical variables
top_cat_vars = categorical_rankings.sort_values("final_score").head(10)["Variable"].tolist()
df_encoded_cat = pd.get_dummies(filtered_cat[top_cat_vars], drop_first=True)

# Merge with selected quantitative variables
X_combined = pd.concat([df_selected_quant, df_encoded_cat], axis=1)

# Explicit conversion to float and deletion of non-numeric columns
X_combined_clean = X_combined.copy()
for col in X_combined.columns:
    try:
        X_combined_clean[col] = X_combined[col].astype(float)
    except:
        X_combined_clean.drop(columns=col, inplace=True)

# Add constant and align with y
X_combined_clean = sm.add_constant(X_combined_clean, has_constant='add')
y_clean = df["SalePrice"]

# Cleaning
valid_index = X_combined_clean.dropna().index.intersection(y_clean.dropna().index)
X_final = X_combined_clean.loc[valid_index]
y_final = y_clean.loc[valid_index]

# Linear Regression
model = sm.OLS(y_final, X_final).fit()
print(model.summary())

# Residual analysis
residuals = model.resid
fitted = model.fittedvalues

plt.figure(figsize=(8, 5))
sns.scatterplot(x=fitted, y=residuals)
plt.axhline(0, color='red', linestyle='--')
plt.title("Résidus vs valeurs ajustées")
plt.xlabel("Valeurs ajustées")
plt.ylabel("Résidus")
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 5))
sns.histplot(residuals, kde=True, color='skyblue')
plt.title("Distribution des résidus")
plt.tight_layout()
plt.show()

sm.qqplot(residuals, line='s')
plt.title("Q-Q Plot des résidus")
plt.tight_layout()
plt.show()


## 5. Time Series Analysis

In [ ]:
# Agregatting month and year sold dates
df_quant['SaleDate'] = pd.to_datetime(dict(year=df_quant['YrSold'], month=df_quant['MoSold'], day=1))

monthly_avg_prices = df_quant.groupby('SaleDate')['SalePrice'].mean().reset_index()


plt.figure(figsize = (8, 5))
plt.plot(monthly_avg_prices['SaleDate'], monthly_avg_prices['SalePrice'], '.', alpha = 0.5, linestyle = "-", color="blue")
plt.xlabel("Sale Date")
plt.ylabel("Sale Price")
plt.title("Average house Sale Prices Over Time")
plt.grid(True)
plt.tight_layout()
plt.show()

result = seasonal_decompose(monthly_avg_prices['SalePrice'], model='additive', period=12)
result.plot()

In [ ]:
# Smoothing (Moving Average)
# Removes short term fluctuations and highlights long term trends

monthly_avg_prices['MA6'] = monthly_avg_prices['SalePrice'].rolling(window=6).mean()
monthly_avg_prices['MA6'].plot() # bleu

monthly_avg_prices['MA12'] = monthly_avg_prices['SalePrice'].rolling(window=12).mean()
monthly_avg_prices['MA12'].plot() # orange
plt.title("Smoothed House Prices Over Time")
plt.legend()

# Both show the average price is slowly going down


In [ ]:
# Stationarity test, ADF
result2 = adfuller(monthly_avg_prices['SalePrice'].dropna())

print(f'ADF Statistic: {result2[0]}, p-value: {result2[1]}')

# if p-value is smaller than 0.05, can model directly

In [ ]:
plot_acf(monthly_avg_prices['SalePrice'].dropna(), lags=20) # lag 20 = arbitrary choice,
plt.grid(True)
# If too small we miss long term patterns, too big too much noise, we compare with almost 2 years before


plot_pacf(monthly_avg_prices['SalePrice'].dropna(), lags=20)
plt.grid(True)

# lag 4 is significant

In [ ]:
# Fitting AR, MA, ARMA models
# AR(4)
x = monthly_avg_prices['SalePrice'].dropna()

ar_model = ARIMA(x, order=(4, 0, 0)).fit()

#MA(4)
ma_model = ARIMA(x, order=(0,0,4)).fit()

#ARMA(4,4)
arma_model = ARIMA(x, order=(4,0,0)).fit()

print("AIC & BIC Scores")
print("AR(4):   AIC =", ar_model.aic, " | BIC =", ar_model.bic)
print("MA(4):   AIC =", ma_model.aic, " | BIC =", ma_model.bic)
print("ARMA(4,4): AIC =", arma_model.aic, " | BIC =", arma_model.bic)

In [ ]:
# Fit ARIMA(4,0,0)
model_ar = ARIMA(x, order=(4, 0, 0)).fit()

# Fit ARIMA(0,0,4)
model_ma = ARIMA(x, order=(0, 0, 4)).fit()

# Fit SARIMAX(1,0,1)(1,0,0,12)
model_sarimax = SARIMAX(x, order=(1, 0, 1), seasonal_order=(1, 0, 0, 12)).fit()
sarima_pred = model_sarimax.predict(start=x.index[0], end=x.index[-1])

# Subplots
fig, axs = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Model Fits Comparison')

# ARIMA(4,0,0)
axs[0, 0].plot(x, label='Observed')
axs[0, 0].plot(model_ar.fittedvalues, label='Fitted', color='orange')
axs[0, 0].set_title('ARIMA(4,0,0)')
axs[0, 0].legend()

# ARIMA(0,0,4)
axs[0, 1].plot(x, label='Observed')
axs[0, 1].plot(model_ma.fittedvalues, label='Fitted', color='orange')
axs[0, 1].set_title('ARIMA(0,0,4)')
axs[0, 1].legend()

# SARIMAX(1,0,1)(1,0,0,12)
axs[1, 0].plot(x, label='Observed')
axs[1, 0].plot(sarima_pred, label='Fitted', color='orange')
axs[1, 0].set_title('SARIMAX(1,0,1)(1,0,0,12)')
axs[1, 0].legend()

# Empty subplot
axs[1, 1].axis('off')

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

# Print AIC and BIC for comparison
print("Model Comparison:")
print(f"ARIMA(4,0,0)    → AIC: {model_ar.aic:.2f}, BIC: {model_ar.bic:.2f}")
print(f"ARIMA(0,0,4)    → AIC: {model_ma.aic:.2f}, BIC: {model_ma.bic:.2f}")
print(f"SARIMAX         → AIC: {model_sarimax.aic:.2f}, BIC: {model_sarimax.bic:.2f}")
